# Vetting Between IFsHistSeries vs New Database one table at one time 

In [ ]:
import pandas as pd
import numpy as np
import sqlite3

In [ ]:
new_conn = sqlite3.connect(r"C:\Users\Norah\Downloads\IFsDataImport (8).db")
hist_conn = sqlite3.connect(r"c:\IFs\DATA\IFsHistSeries.db")

In [ ]:
#puller is the new data, hist is old data
tbname = 'SeriesFreedomEcon'

dt_puller = pd.read_sql_query(f"SELECT * FROM '{tbname}'", new_conn)
col = dt_puller.columns[dt_puller.dtypes.eq('float64')]

dt_hist = pd.read_sql_query(f"SELECT * FROM '{tbname}'", hist_conn)

dt_hist = dt_hist.melt(id_vars= ["Country","FIPS_CODE"],var_name="Year",value_name="New_Value")
dt_puller = dt_puller.melt(id_vars= ["Country","FIPS_CODE"],var_name="Year",value_name="IFs_Value")
dt_puller = dt_puller.drop_duplicates()
dt_hist = dt_hist.drop_duplicates()
dt_hist = dt_hist.sort_values(by=['Country', 'FIPS_CODE', 'Year']).reset_index(drop=True)
dt_puller = dt_puller.sort_values(by=['Country', 'FIPS_CODE', 'Year']).reset_index(drop=True)


dt_puller.Year = dt_puller.Year.apply(pd.to_numeric, errors='coerce')
dt_hist.Year = dt_hist.Year.apply(pd.to_numeric, errors='coerce')
dt_puller.IFs_Value = dt_puller.IFs_Value.apply(pd.to_numeric, errors='coerce')
dt_hist.New_Value = dt_hist.New_Value.apply(pd.to_numeric, errors='coerce')

compare = dt_hist.merge(
                       dt_puller, 
                       left_on =[
                                 "Country",
                                 "FIPS_CODE",
                                 "Year",
                                 ], 
                       right_on =[
                                  "Country",
                                  "FIPS_CODE",
                                  "Year",
                                  ],
                       how = 'outer', 
                       )
compare = compare[~compare[['New_Value','IFs_Value']].isna().all(axis=1)]
compare = compare[compare['Year'] <= 2027]
compare['check'] = abs(compare['New_Value'] - compare['IFs_Value'])/compare['IFs_Value']
compare[((compare['check'] >= 0.15) | (compare['check'].isnull()))
        ]

compare

# can check for N/A values or values above a certain threshold like 0 
#comparech = compare[compare['check'].isna()]
#comparech = compare[compare['check'] > 0]
#comparech

# to Excel
#compare.to_csv('GDPcomparison2011new.csv')


If you want to see full list in python itself and not export to excel

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

compare
